# TensorflowでXorを作成

訓練データと正解ラベル

In [1]:
import numpy as np
#訓練データと正解ラベル
train = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
label = np.array([[0],
                              [1],
                              [1],
                              [0]])

## 1：モデルの定義

In [2]:
#モデルの定義
import tensorflow as tf

class MLP(tf.keras.Model):
    #多重パーセプトロン
    def __init__(self, input_dim, hidden_dim, output_dim):
        # input_dim: 入力する1データあたりの値の形状
        # hidden_dim(int): 隠れ層のニューロン数
        # output_dim(int): 出力層のニューロン数
        
        super(MLP, self).__init__()
        #l1: 隠れ層
        self.l1 = tf.keras.layers.Dense(
            units=hidden_dim,   # ニューロンのサイズ
            input_dim=input_dim, # 入力データの形状
            activation="sigmoid")  # 活性化関数はシグモイド
        #l2: 出力層
        self.l2 = tf.keras.layers.Dense(
            units=output_dim,# ニューロンのサイズ
            activation="sigmoid") # 活性化関数はシグモイド
        
    @tf.function
    def call(self, x, training=None):
        # x(ndarray(float32)): 訓練データまたは検証データ
        # training(bool): 訓練True、検証False
        # Returns(foalt32): 出力層からの出力値
        h = self.l1(x) # 隠れ層の出力
        y = self.l2(h) #出力層の出力
        return y

# 入力層2ニューロン、隠れ層2ニューロン、出力層1ニューロンのモデルを生成
model = MLP(2, 2, 1)


## 2：損失関数とオプティマイザーの生成

In [3]:
#バイナリ用のクロスエントロピー誤差のオブジェクトを生成
loss_fn = tf.keras.losses.BinaryCrossentropy()
#勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)

## 4:バックプロバゲーション(逆誤差伝播法）を実行する関数の定義

### 勾配降下アルゴリズムによるパラメーターの更新処理

In [4]:
@tf.function
def train_step(x,t):
    # x(ndarray(float32)): 訓練データ
    # t(ndarray(float32)): 正解ラベル
    #　Returns：　MLPの出力と正解ラベルのクロスエントロピー誤差
    
    # 自動微分による勾配計算のための操作を記録するブロック
    with tf.GradientTape() as tape:
        predictions = model(x) #モデルに入力して順伝播の出力値を取得
        pred_loss = loss_fn(t,predictions) # 出力値と正解ラベルの誤差を取得
    
    # tapeに記録された操作を使用して誤差の勾配を計算
    gradients = tape.gradient(
          pred_loss, # 現在の誤差
          model.trainable_variables) # 更新可能なバイアス、重みのリストを取得
    
    # 勾配降下法の更新式を適用してバイアス、重みを更新
    optimizer.apply_gradients(
    zip(gradients, # 取得済みの勾配
           model.trainable_variables)) # 更新可能なバイアス、重みのリスト
    
    return pred_loss

## 5：モデルを定義して学習する

In [5]:
#　エポック数
epochs = 4000

for epoch in range(epochs):
    # 1エポックごとの損失を保持する変数
    epoch_loss = 0
    
    # データをモデルに入力し、バイアス、重みを更新して誤差を取得
    loss = train_step(train, label)
    epoch_loss += loss.numpy()
    
    # 1000エポックごとに結果を出力
    if (epoch+1) % 1000 == 0:
        print("epoch({}) loss: {:.4f}".format(epoch+1,epoch_loss))
        
# モデルの構造を出力
model.summary()

epoch(1000) loss: 0.1248
epoch(2000) loss: 0.0183
epoch(3000) loss: 0.0095
epoch(4000) loss: 0.0064
Model: "mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  6         
_________________________________________________________________
dense_1 (Dense)              multiple                  3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


## 6:学習結果の確認

In [6]:
print(model(train))
print(tf.cast(((model(train)) >= 0.5), tf.int32))

tf.Tensor(
[[0.00530535]
 [0.9930601 ]
 [0.99307287]
 [0.00615308]], shape=(4, 1), dtype=float32)
tf.Tensor(
[[0]
 [1]
 [1]
 [0]], shape=(4, 1), dtype=int32)


## この結果から、Xorが実現できていることが分かる。